### All needed imports

In [1]:
import os
import cv2
import numpy as np

import tensorflow as tf
assert tf.__version__.startswith('2')
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator



/home/sylvia/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Limit GPU Usage

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(gpus[0], [tf.config.LogicalDeviceConfiguration(memory_limit=2048)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-07-31 15:23:03.749333: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-31 15:23:03.765224: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-31 15:23:03.765371: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-07-31 15:23:03.766263: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Preprocessing Data for Model 1

#### Video Paths

In [3]:
falling_paths = [
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall2.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall3.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall4.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall1.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall5.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall6.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall7.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall8.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall9.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall10.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall11.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall12.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall13.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall14.mp4",
                 "./../datasets/vids/splitted/new_moving/resized_logitech-fall15.mp4"
                 ]

default_paths = [
                 "./../datasets/vids/splitted/new_still/resized_logitech-default1.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default2.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default3.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default4.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default5.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default6.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default7.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default8.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default9.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default10.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default11.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default12.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default13.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default14.mp4",
                 "./../datasets/vids/splitted/new_still/resized_logitech-default15.mp4"
                 ]

#### Processing code, returns frame (224,224,3) and frame_diffs

In [16]:
def process_videos(video_paths, label):
    frames = []
    frame_diffs = []
    labels = []
    
    for path in video_paths:
        video_cap = cv2.VideoCapture(path)
        
        prev_gray_frame = None
        
        while video_cap.isOpened():
            ret, frame = video_cap.read()
            
            if not ret:
                break
            
            # Resize and convert frame to RGB
            frame_resized = cv2.resize(frame, (224, 224))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
            
            # Convert frame to grayscale
            gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
            
            if prev_gray_frame is None:
                # frame_diff = 0
                frame_avg_diff = 0
            else:
                # frame_diff = np.mean(np.abs(gray_frame - prev_gray_frame))
                frame_diff = cv2.absdiff(prev_gray_frame, gray_frame)
                total_sum = np.sum(frame_diff)
                frame_avg_diff = total_sum / (224 * 224)
            
            prev_gray_frame = gray_frame
            
            frames.append(frame_rgb)
            frame_diffs.append(frame_avg_diff)
            labels.append(label)  # 0 for still, 1 for moving
            
            
        video_cap.release()

    return np.array(frames), np.array(frame_diffs), np.array(labels)

In [5]:
# Reshaping to 224 x 224 x 4 for the convolutional model
def combine_frames_and_diffs(frames, frame_diffs):
    frame_diffs_expanded = frame_diffs.reshape(-1, 224, 224, 1) # Reshape to match frame dimensions
    combined_input = np.concatenate([frames, frame_diffs_expanded], axis=-1)
    return combined_input

In [19]:
# Generate the frames and frame_diffs array for still
still_frames, still_diff, still_labels = process_videos(default_paths, 0)
print(still_frames.shape)
print(still_diff.shape)
print(still_labels.shape)

# checking data of still_diff
# for x in still_diff:
#     print(x)

# print(sum(still_diff)/still_diff.shape) # Using mean calculation: 83.18836141

falling_frames, falling_diff, falling_labels = process_videos(falling_paths, 0)
print(falling_frames.shape)
print(falling_diff.shape)
print(falling_labels.shape)

# for x in falling_diff:
#     print(x)

# print(sum(falling_diff)/falling_diff.shape) # Using mean calculation: 92.13105689

# Think about whether the difference should be per pixel instead of per frame

(567, 224, 224, 3)
(567,)
(567,)
(513, 224, 224, 3)
(513,)
(513,)
0.0
0.07266422193877552
8.697883450255102
0.04077646683673469
5.883450255102041
12.674744897959183
33.86214524872449
31.303531568877553
43.267657844387756
42.511639030612244
57.384984853316325
57.45426100127551
46.97652264030612
35.47552614795919
37.77558992346939
21.624621332908163
80.5592115752551
42.250119579081634
0.41599569515306123
0.46876992984693877
5.779296875
55.569415656887756
97.87846779336735
0.0034478635204081634
83.90921954719387
0.001295440051020408
29.744100765306122
21.931062659438776
0.12250876913265306
9.958685427295919
17.55986926020408
0.0
22.034060108418366
0.0
0.022301498724489797
17.873007015306122
0.3237802933673469
45.771464445153065
0.05630181760204082
68.83464205994898
52.84281329719388
54.16240832270408
55.327168367346935
19.02638711734694
20.154675542091837
25.786073022959183
37.66216916454081
29.561264349489797
52.704021843112244
132.8995137117347
0.040935905612244895
195.9503547512755
57.